## Load Dataset

In [2]:
!wget -O dataset_train.csv \
https://raw.githubusercontent.com/thoriqtau/BILSTM-Attention-CRF-for-Indonesian-Profanity/main/dataset/dataset_train.csv -qq

!wget -O dataset_test.csv \
https://raw.githubusercontent.com/thoriqtau/BILSTM-Attention-CRF-for-Indonesian-Profanity/main/dataset/dataset_test.csv -qq


In [3]:
import pandas as pd

# Read dataset train
df_train = pd.read_csv("/content/dataset_train.csv", header=None)
train_data_list = df_train.iloc[:, 0].tolist()

# Read dataet Test
df_test = pd.read_csv("//content/dataset_test.csv", header=None)
test_data_list = df_test.iloc[:, 0].tolist()

In [4]:
print("Data Train:\n", train_data_list[0], "\n")
print("Data Train:\n", test_data_list[0])

Data Train:
 lucunya/O adalah/O kak/O pacar/O nyadain/O ini/O telat/O bgt/O guys/O gue/O rasa/O pengaruh/O mecin/O tuh/O emang/O bikin/O kita/O slow/O respon/O agak/O bolot/FIS-POS gitu/O  

Data Train:
 loe/O lantas/O remehkan/O perhatian/O yang/O gue/O kasih/O khusus/O ke/O elo/O basic/O elo/O cowok/O bego/PER-POS


## Parsing Sentence

In [5]:
# Parsing sentence and label function
def parse_inline_dataset(dataset):
    sentences = []
    labels = []
    for line in dataset:
        tokens = line.strip().split()
        sentence = []
        label_seq = []
        for token in tokens:
            if '/' in token:
                word, tag = token.rsplit('/', 1)
                sentence.append(word.lower())
                label_seq.append(tag)
        sentences.append(sentence)
        labels.append(label_seq)
    return sentences, labels

In [6]:
all_train_sentences, all_train_labels = parse_inline_dataset(train_data_list)
all_test_sentences, all_test_labels = parse_inline_dataset(test_data_list)

In [7]:
print("Token Train Sentences:\n", all_train_sentences[0], "\n")
print("Token Train Labels:\n", all_train_labels[0])

print("--------------------------------------------")

print("Token Test Sentences:\n", all_test_sentences[0], "\n")
print("Token Test Labels:\n", all_test_labels[0])

Token Train Sentences:
 ['lucunya', 'adalah', 'kak', 'pacar', 'nyadain', 'ini', 'telat', 'bgt', 'guys', 'gue', 'rasa', 'pengaruh', 'mecin', 'tuh', 'emang', 'bikin', 'kita', 'slow', 'respon', 'agak', 'bolot', 'gitu'] 

Token Train Labels:
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'FIS-POS', 'O']
--------------------------------------------
Token Test Sentences:
 ['loe', 'lantas', 'remehkan', 'perhatian', 'yang', 'gue', 'kasih', 'khusus', 'ke', 'elo', 'basic', 'elo', 'cowok', 'bego'] 

Token Test Labels:
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PER-POS']


## Build Vocab

In [8]:
# Vocabulary mapping function
def build_vocab(sequences, specials=["<PAD>", "<UNK>"]):
    vocab = {tok: idx for idx, tok in enumerate(specials)}
    for seq in sequences:
        for item in seq:
            if item not in vocab:
                vocab[item] = len(vocab)
    return vocab

In [9]:
# wrod to index vocab
word_to_ix = build_vocab(all_train_sentences, specials=["<PAD>", "<UNK>"])
# tag to index vocab
tag_to_ix = build_vocab(all_train_labels, specials=["<PAD>"])

# index to word vocab
ix_to_word = {idx: word for word, idx in word_to_ix.items()}
# index to tag vocab
ix_to_tag = {idx: tag for tag, idx in tag_to_ix.items()}

In [10]:
print("Word_to_ix (see 5):\n", list(word_to_ix.items())[:5], "\n")
print("Tag_to_ix (see 5):\n", list(tag_to_ix.items())[:5], "\n")

print("--------------------------------------------")

print("Ix_to_word (see 5):\n", list(ix_to_word.items())[:5], "\n")
print("Ix_to_tag (see 5):\n", list(ix_to_tag.items())[:5], "\n")

Word_to_ix (see 5):
 [('<PAD>', 0), ('<UNK>', 1), ('lucunya', 2), ('adalah', 3), ('kak', 4)] 

Tag_to_ix (see 5):
 [('<PAD>', 0), ('O', 1), ('FIS-POS', 2), ('PER-NEG', 3), ('SEKS-NEG', 4)] 

--------------------------------------------
Ix_to_word (see 5):
 [(0, '<PAD>'), (1, '<UNK>'), (2, 'lucunya'), (3, 'adalah'), (4, 'kak')] 

Ix_to_tag (see 5):
 [(0, '<PAD>'), (1, 'O'), (2, 'FIS-POS'), (3, 'PER-NEG'), (4, 'SEKS-NEG')] 



In [11]:
X_train = [[word_to_ix.get(w, word_to_ix["<UNK>"]) for w in sent] for sent in all_train_sentences]
y_train = [[tag_to_ix[l] for l in labels] for labels in all_train_labels]

X_test = [[word_to_ix.get(w, word_to_ix["<UNK>"]) for w in sent] for sent in all_test_sentences]
y_test = [[tag_to_ix[l] for l in labs] for labs in all_test_labels]

In [12]:
print("X_train (see 1):\n", all_train_sentences[0], "\n" ,X_train[0], "\n")
print("y_train (see 1):\n", all_train_labels[0], "\n" ,y_train[0], "\n")

print("--------------------------------------------")

print("X_test (see 1):\n", all_train_sentences[0], "\n", X_test[0], "\n")
print("y_test (see 1):\n", all_train_labels[0], "\n", y_test[0], "\n")

X_train (see 1):
 ['lucunya', 'adalah', 'kak', 'pacar', 'nyadain', 'ini', 'telat', 'bgt', 'guys', 'gue', 'rasa', 'pengaruh', 'mecin', 'tuh', 'emang', 'bikin', 'kita', 'slow', 'respon', 'agak', 'bolot', 'gitu'] 
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23] 

y_train (see 1):
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'FIS-POS', 'O'] 
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1] 

--------------------------------------------
X_test (see 1):
 ['lucunya', 'adalah', 'kak', 'pacar', 'nyadain', 'ini', 'telat', 'bgt', 'guys', 'gue', 'rasa', 'pengaruh', 'mecin', 'tuh', 'emang', 'bikin', 'kita', 'slow', 'respon', 'agak', 'bolot', 'gitu'] 
 [341, 1, 1, 1196, 60, 11, 1197, 1141, 215, 1, 1, 1, 75, 397] 

y_test (see 1):
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'FIS-POS', 'O'] 
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8] 


## Dataloader

In [13]:
import torch
from torch.utils.data import Dataset

# Create class dataloader
class NERDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.long),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

In [14]:
from torch.nn.utils.rnn import pad_sequence

# Padding function
def collate_fn(batch, pad_word_ix, pad_tag_ix):
    sentences, labels = zip(*batch)

    sentences = list(sentences)
    labels = list(labels)

    lengths = torch.tensor([len(s) for s in sentences])

    padded_sentences = pad_sequence(
        sentences,
        batch_first=True,
        padding_value=pad_word_ix
    )

    padded_labels = pad_sequence(
        labels,
        batch_first=True,
        padding_value=pad_tag_ix
    )

    return padded_sentences, padded_labels, lengths

In [15]:
from torch.utils.data import DataLoader
from functools import partial

train_dataset = NERDataset(X_train, y_train)
test_dataset = NERDataset(X_test, y_test)

collate_with_pad = partial(
    collate_fn,
    pad_word_ix=word_to_ix["<PAD>"],
    pad_tag_ix=tag_to_ix["<PAD>"]
)

train_loader = DataLoader(
    train_dataset,
    batch_size=8, # Batch size 8
    shuffle=True, # Shuffle True
    collate_fn=collate_with_pad
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8, # Batch size 8
    shuffle=False, # Shuffle False
    collate_fn=collate_with_pad
)


In [16]:
# ix_to_word for tansform index to word
ix_to_word = {v: k for k, v in word_to_ix.items()}

sentences = []
for X_batch, y_batch, lengths in train_loader:
    for i in range(X_batch.size(0)):
        tokens = [
            ix_to_word[X_batch[i, j].item()]
            for j in range(X_batch.size(1))  # full length, include PAD
        ]
        sentence = " ".join(tokens)
        sentences.append(sentence)

In [17]:
print("X_train (see 5):")
for sentence in sentences[:5]:
  print(" ", sentence)


X_train (see 5):
  yang juara ngibul kan junjungan kalian wahai bani kampret <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
  sini om jilatin memek nya mau ngga <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
  patung anjing hachiko di jepang menjadi simbol kesetiaan abadi <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
  kapal kita karam monyet kasihan patah hati <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
  kirain budek <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Bi-LSTM Arsitektur

In [18]:
# install pytorch-crf
!pip install pytorch-crf -qq

In [19]:
import torch
from torchcrf import CRF

# Bi-LSTM - Attention - CRF architecture
class BiLSTM_Attend_CRF(torch.nn.Module):
    def __init__(self, pad_ix, tag_size, vocab_size, hidden_size, embedding_dim, heads=1):
        super().__init__()

        self.hidden_size = hidden_size

        # Embedding layer: maps token indices to dense vectors
        # padding_idx ensures the padding token is ignored during training
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_ix)

        # Bidirectional LSTM to capture left and right context
        # hidden_size // 2 is used because of bidirectionality
        self.bi_lstm = torch.nn.LSTM(
            embedding_dim,
            hidden_size // 2,
            batch_first=True,
            bidirectional=True
        )

        # Multi-head self-attention layer
        # Allows the model to focus on important tokens in the sequence
        self.multiheadattention = torch.nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=heads,
            dropout=0.1,
            batch_first=True
        )
        # Linear layer to project hidden states to tag scores (emissions)
        self.hidden2tag = torch.nn.Linear(hidden_size, tag_size)
        # CRF layer to model dependencies between output tags
        self.crf = CRF(num_tags=tag_size, batch_first=True)
        # Layer normalization for training stability
        self.layer_norm = torch.nn.LayerNorm(hidden_size)

    def init_hidden(self, batch_size, device):
        # Initialize hidden and cell states for BiLSTM
        # 2 corresponds to bidirectional LSTM
        return (
            torch.randn(2, batch_size, self.hidden_size // 2, device=device),
            torch.randn(2, batch_size, self.hidden_size // 2, device=device)
        )

    def forward(self, inp):
        # inp shape: (batch_size, sequence_length)
        batch_size = inp.size(0)
        device = inp.device

        # Convert token indices to embeddings
        embeds = self.embedding(inp)

        # Initialize LSTM hidden states
        hidden = self.init_hidden(batch_size, device)

        # BiLSTM forward pass
        lstm_out, _ = self.bi_lstm(embeds, hidden)

        # Normalize LSTM outputs
        lstm_out = self.layer_norm(lstm_out)

        # Self-attention mechanism
        # Query, key, and value are all LSTM outputs
        attn_output, _ = self.multiheadattention(lstm_out, lstm_out, lstm_out)

        # Residual connection to preserve original LSTM information
        attn_output = attn_output + lstm_out

        # Project features to tag space (emission scores)
        logits = self.hidden2tag(attn_output)
        return logits

    def crf_neg_log_likelihood(self, inp, tags, mask=None):
        # Compute emission scores
        logits = self.forward(inp)

        # Create mask to ignore padding tokens if not provided
        if mask is None:
            pad = torch.tensor(0, device=tags.device)
            mask = tags.ne(pad)
        # Negative log-likelihood loss for CRF
        return -self.crf(logits, tags, mask=mask, reduction='sum')

    def crf_decode(self, inp, mask=None):
        # Compute emission scores
        logits = self.forward(inp)

        # Create mask based on input padding if not provided
        if mask is None:
            pad = torch.tensor(0, device=inp.device)
            mask = inp.ne(pad)
        # Decode the best tag sequence using Viterbi algorithm
        return self.crf.decode(logits, mask=mask)


## Train step function

In [20]:
def train_step(model, dataloader, optimizer, device):
    # Set model to training mode
    model.train()
    total_loss = 0

    # Iterate over batches from the dataloader
    # Each batch contains input tokens, and labels
    for X_batch, y_batch, _ in dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # Create mask: True for valid (non-padding) tokens
        # Padding token is assumed to have index 0
        mask = X_batch.ne(0)

        # Reset gradients from the previous step
        optimizer.zero_grad()

        # Compute negative log-likelihood loss from the CRF layer
        loss = model.crf_neg_log_likelihood(
            inp=X_batch,
            tags=y_batch,
            mask=mask
        )

        # Backpropagate gradients
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate batch loss
        total_loss += loss.item()
    # Return average loss per batch
    return total_loss / len(dataloader)

# Test step function

In [21]:
def test_step(model, dataloader, device):
    # Set model to evaluation mode
    model.eval()

    total_loss = 0
    all_preds = []    # Store predicted tag sequences
    all_labels = []   # Store ground-truth tag sequences

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for X_batch, y_batch, _ in dataloader:
            # Move batch data to the selected device
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            # Create mask to ignore padding tokens
            mask = X_batch.ne(0)

            # Compute CRF negative log-likelihood loss
            loss = model.crf_neg_log_likelihood(
                inp=X_batch,
                tags=y_batch,
                mask=mask
            )
            total_loss += loss.item()

            # Decode the best tag sequence using CRF (Viterbi decoding)
            predictions = model.crf_decode(
                X_batch,
                mask=mask
            )

            # Collect predictions and labels without padding
            for pred_seq, true_seq, m in zip(predictions, y_batch, mask):
                # Number of valid (non-padding) tokens
                valid_len = m.sum().item()

                # Truncate predictions and labels to valid length
                all_preds.append(pred_seq[:valid_len])
                all_labels.append(true_seq[:valid_len].cpu().tolist())

    # Compute average loss per batch
    avg_loss = total_loss / len(dataloader)
    return avg_loss, all_preds, all_labels

## Dictionary Entity

In [22]:
def entity_dict(tag_to_ix):
    # Define entity labels to be evaluated (exclude "O" and non-entity tags)
    entity_labels = [
        tag_to_ix["PER-POS"],   # Person - Positive
        tag_to_ix["PER-NEG"],   # Person - Negative
        tag_to_ix["HWN-POS"],   # Animal - Positive
        tag_to_ix["HWN-NEG"],   # Animal - Negative
        tag_to_ix["FIS-POS"],   # Physical - Positive
        tag_to_ix["FIS-NEG"],   # Physical - Negative
        tag_to_ix["SEKS-NEG"],  # Sexual - Negative
    ]

    # Initialize metric containers for each entity label
    metrics = {
        label: {
            "TP": 0,            # True Positives
            "FP": 0,            # False Positives
            "FN": 0,            # False Negatives
            "Precision": 0.0,   # Precision score
            "Recall": 0.0,      # Recall score
            "F1-Score": 0.0,    # F1-score
            "Support": 0,       # Number of true occurrences
        }
        for label in entity_labels
    }
    return metrics, entity_labels

## Metric Function

In [23]:
from sklearn.metrics import precision_recall_fscore_support
import copy

# Compute precision score
def precision_score(tp, fp):
  return tp / (tp + fp) if (tp + fp) > 0 else 0.0

# Compute recall score
def recall_score(tp, fn):
    return tp / (tp + fn) if (tp + fn) > 0 else 0.0

# Compute F1-score from precision and recall
def f1_score(p, r):
    return 2 * p * r / (p + r) if (p + r) > 0 else 0.0

# Flatten a list of sequences into a single list
def flatten(seqs):
    return [x for seq in seqs for x in seq]

# Compute TP, FP, and FN for each entity label (token-level)
def compute_tp_fp_fn(preds, labels, entity_score_dict, entity_labels):
    y_true = flatten(labels)
    y_pred = flatten(preds)

    # Deep copy to avoid modifying the original dictionary
    score_dict = copy.deepcopy(entity_score_dict)

    # Iterate over each predicted and true label pair
    for t, p in zip(y_true, y_pred):
        # Count support for each entity
        if t in entity_labels:
            score_dict[t]["Support"] += 1
        # Update True Positive and False Positive counts
        if p in entity_labels:
            if t == p:
                score_dict[p]["TP"] += 1  # True Positive
            else:
                score_dict[p]["FP"] += 1  # False Positive
        # Update False Negative count
        if t in entity_labels and t != p:
            score_dict[t]["FN"] += 1  # False Negative

    # Compute precision, recall, and F1-score for each entity label
    for label in entity_labels:
        tp = score_dict[label]["TP"]
        fp = score_dict[label]["FP"]
        fn = score_dict[label]["FN"]

        p = precision_score(tp, fp)
        r = recall_score(tp, fn)
        f1 = f1_score(p, r)

        score_dict[label]["Precision"] = p
        score_dict[label]["Recall"] = r
        score_dict[label]["F1-Score"] = f1
    return score_dict

# Compute micro-averaged token-level precision, recall, and F1-score
def compute_token_f1(preds, labels, entity_labels):
    # Flatten predicted and true label sequences
    y_true = flatten(labels)
    y_pred = flatten(preds)

    # Compute micro-averaged metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true,
        y_pred,
        labels=entity_labels,
        average="micro",
        zero_division=0
    )
    return precision, recall, f1

In [24]:
def train_model(
    model,
    checkpoints,
    epochs,
    train_loader,
    test_loader,
    optimizer,
    device,
    entity_metric_dict,
    entity_labels,
):
    best_metric = 0.0

    # Initialize entity-level metric containers
    entity_metric = entity_metric_dict
    entity = entity_labels

    # Training loop over epochs
    for epoch in range(epochs):
        # Perform one training epoch
        train_loss = train_step(model, train_loader, optimizer, device)

        # Evaluate model
        test_loss, preds, labels = test_step(model, test_loader, device)

        # Compute TP, FP, and FN for each entity label
        score_dict = compute_tp_fp_fn(preds, labels, entity_metric, entity)

        # Compute overall token-level precision, recall, and F1-score
        precision, recall, f1 = compute_token_f1(preds, labels, entity)

        print(f"{'='*80}")
        print(f"EPOCH [{epoch+1}/{epochs}]")
        print(f"{'='*80}")
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {test_loss:.4f}")
        print(
            f"Overall - Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}"
        )

        # Save the model checkpoint if current F1-score is the best
        if f1 > best_metric:
            best_metric = f1
            checkpoint = {
                "model_state_dict": model.state_dict(),
                "pad_ix": checkpoints["pad_ix"],
                "tag_size": checkpoints["tag_size"],
                "vocab_size": checkpoints["vocab_size"],
                "hidden_size": checkpoints["hidden_size"],
                "embedding_dim": checkpoints["embedding_dim"],
                "heads": checkpoints["heads"],
                "word_to_ix": checkpoints["word_to_ix"],
                "tag_to_ix": checkpoints["tag_to_ix"],
                "ix_to_word": checkpoints["ix_to_word"],
                "ix_to_tag": checkpoints["ix_to_tag"],
            }
            torch.save(checkpoint, "best_model.sav")
    print(f"✔ Model terbaik disimpan dengan F1-Score: {best_metric:.4f}\n")
    return score_dict

## Initialize Model

In [25]:
checkpoint = {
    "pad_ix": word_to_ix["<PAD>"],
    "tag_size": len(tag_to_ix),
    "vocab_size": len(word_to_ix),
    "hidden_size": 256,
    "embedding_dim": 200,
    "heads": 4,
    "word_to_ix": word_to_ix,
    "tag_to_ix": tag_to_ix,
    "ix_to_word": ix_to_word,
    "ix_to_tag": ix_to_tag,
}

In [26]:
# Setting default device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BiLSTM-Attention-CRF model
model = BiLSTM_Attend_CRF(
    pad_ix=checkpoint["pad_ix"],
    tag_size=checkpoint["tag_size"],
    vocab_size=checkpoint["vocab_size"],
    hidden_size=checkpoint["hidden_size"],
    embedding_dim=checkpoint["embedding_dim"],
    heads=checkpoint["heads"],
).to(device)

# AdamW optimizer and learning rate 0.001
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [27]:
epochs = 200
entity_metric_dict, entity_labels = entity_dict(tag_to_ix)

score_dict = train_model(
    model=model,
    checkpoints=checkpoint,
    epochs=epochs,
    train_loader=train_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=device,
    entity_metric_dict=entity_metric_dict,
    entity_labels=entity_labels,
)

EPOCH [1/200]
Train Loss: 38.5256 | Val Loss: 32.2159
Overall - Precision: 0.7692 | Recall: 0.4225 | F1: 0.5455
EPOCH [2/200]
Train Loss: 9.0036 | Val Loss: 26.5299
Overall - Precision: 0.7273 | Recall: 0.5634 | F1: 0.6349
EPOCH [3/200]
Train Loss: 3.2707 | Val Loss: 31.3580
Overall - Precision: 0.6964 | Recall: 0.5493 | F1: 0.6142
EPOCH [4/200]
Train Loss: 1.7904 | Val Loss: 34.8335
Overall - Precision: 0.6429 | Recall: 0.5070 | F1: 0.5669
EPOCH [5/200]
Train Loss: 0.9766 | Val Loss: 43.3547
Overall - Precision: 0.6964 | Recall: 0.5493 | F1: 0.6142
EPOCH [6/200]
Train Loss: 0.4925 | Val Loss: 44.1848
Overall - Precision: 0.7455 | Recall: 0.5775 | F1: 0.6508
EPOCH [7/200]
Train Loss: 0.2204 | Val Loss: 47.8643
Overall - Precision: 0.7091 | Recall: 0.5493 | F1: 0.6190
EPOCH [8/200]
Train Loss: 0.1778 | Val Loss: 50.4850
Overall - Precision: 0.7500 | Recall: 0.5915 | F1: 0.6614
EPOCH [9/200]
Train Loss: 0.2032 | Val Loss: 46.0826
Overall - Precision: 0.7500 | Recall: 0.5915 | F1: 0.6614


In [28]:
def entity_score_to_df(entity_score_dict, ix_to_tag):
    rows = []

    for label_ix, scores in entity_score_dict.items():
        rows.append({
            "Entity": ix_to_tag[label_ix],
            "TP": scores["TP"],
            "FP": scores["FP"],
            "FN": scores["FN"],
            "Precision": round(scores["Precision"], 4),
            "Recall": round(scores["Recall"], 4),
            "F1-Score": round(scores["F1-Score"], 4),
            "Support": scores["Support"]
        })

    df = pd.DataFrame(rows)

    # Urutkan biar rapi
    df = df.sort_values("Entity").reset_index(drop=True)

    return df

In [29]:
result = entity_score_to_df(score_dict, ix_to_tag)
result

,Entity,TP,FP,FN,Precision,Recall,F1-Score,Support
0,FIS-NEG,2,3,2,0.4000,0.5000,0.4444,4
1,FIS-POS,0,2,2,0.0000,0.0000,0.0000,2
2,HWN-NEG,8,4,2,0.6667,0.8000,0.7273,10
3,HWN-POS,1,2,2,0.3333,0.3333,0.3333,3
4,PER-NEG,10,4,4,0.7143,0.7143,0.7143,14
5,PER-POS,0,1,3,0.0000,0.0000,0.0000,3
6,SEKS-NEG,21,0,14,1.0000,0.6000,0.7500,35


## Sanity Check

In [30]:
checkpoint = torch.load("best_model.sav", map_location=device)

reloaded_model = BiLSTM_Attend_CRF(
    checkpoint["pad_ix"],
    checkpoint["tag_size"],
    checkpoint["vocab_size"],
    checkpoint["hidden_size"],
    checkpoint["embedding_dim"],
    heads=checkpoint["heads"]
).to(device)

reloaded_model.load_state_dict(checkpoint["model_state_dict"])

print("✔ Model successfully loaded")

✔ Model successfully loaded


In [31]:
def predict_sentence(sentence, model, word_to_ix, ix_to_tag, device):
    # Set model to evaluation mode
    model.eval()

    # Tokenization
    tokens = sentence.lower().split()

    # Map tokens to indices (use <UNK> for unseen words)
    indexed_tokens = [
        word_to_ix.get(tok, word_to_ix["<UNK>"])
        for tok in tokens
    ]

    # Convert to tensor and add batch dimension
    # Shape: (1, sequence_length)
    input_tensor = torch.tensor(
        indexed_tokens,
        dtype=torch.long
    ).unsqueeze(0).to(device)

    # Create mask to ignore padding tokens
    mask = input_tensor.ne(0)

    # Decode the best tag sequence using CRF
    with torch.no_grad():
        predicted_tags = model.crf_decode(
            inp=input_tensor,
            mask=mask
        )[0]

    # Convert tag indices back to label names
    predicted_labels = [
        ix_to_tag[tag_ix] for tag_ix in predicted_tags
    ]
    # Return token-label pairs
    return list(zip(tokens, predicted_labels))


In [32]:
sentence = "Jangan gitu anjing lah"

result = predict_sentence(
    sentence=sentence,
    model=reloaded_model,
    word_to_ix=word_to_ix,
    ix_to_tag=ix_to_tag,
    device=device
)

for word, label in result:
    print(f"{word:<12} -> {label}")

jangan       -> O
gitu         -> O
anjing       -> HWN-NEG
lah          -> O
